In [11]:
import numpy as np
import sklearn as sk
import pandas as pd
import sqlite3
import pickle

In [12]:
f = open('df', 'rb')
df = pickle.load(f)

In [14]:
def feat_pass_yrds(row):
    return row.IsPass * row.Yards

def feat_rush_yrds(row):
    return row.IsRush * row.Yards

def feat_pass_tds(row):
    return row.IsPass * row.IsTouchdown

def feat_rush_tds(row):
    return row.IsRush * row.IsTouchdown

In [15]:
def get_off_stats(filename):
    nfl = pd.read_csv(filename)
    var_list = ['GameDate', 'Quarter', 'Minute', 'Second', 'OffenseTeam',
       'DefenseTeam', 'Down', 'ToGo', 'YardLine', 'Description',
       'SeasonYear', 'Yards', 'Formation',
       'PlayType', 'IsRush', 'IsPass', 'IsIncomplete', 'IsTouchdown',
       'PassType', 'IsSack']

    nfl = nfl[var_list]
    nfl = nfl.dropna(subset=['OffenseTeam'])
    
    nfl['PassYrds'] = nfl.apply(feat_pass_yrds, axis=1)
    nfl['RushYrds'] = nfl.apply(feat_rush_yrds, axis=1)
    nfl['PassTDs'] = nfl.apply(feat_pass_tds, axis=1)
    nfl['RushTDs'] = nfl.apply(feat_rush_tds, axis=1)
    
    off_vars = ['SeasonYear', 'Yards', 'IsRush', 'IsPass', 'IsIncomplete', 
            'PassYrds', 'RushYrds', 'PassTDs', 'RushTDs', 'IsTouchdown']
    
    teams_off_counts = nfl.groupby('OffenseTeam').size()
    teams_off = nfl.groupby('OffenseTeam')
    teams_off = pd.DataFrame(teams_off.agg({'SeasonYear': 'first', 'Yards': 'sum', 'IsRush': 'sum', 'IsPass': 'sum', 
                                        'IsIncomplete': 'sum', 'PassYrds': 'sum', 'RushYrds': 'sum', 'PassTDs': 'sum', 
                                        'RushTDs': 'sum', 'IsTouchdown': 'sum'})[off_vars])
    teams_off['Plays'] = teams_off_counts
    teams_off.reset_index(level=0, inplace=True)
    teams_off = teams_off.rename(columns={'OffenseTeam': 'Team', 'SeasonYear': 'Season', 'IsRush': 'Rushes', 'IsPass': 'Passes', 
                                      'IsIncomplete': 'Incompletes', 'IsTouchdown': 'TDs'})
    
    return teams_off

In [16]:
#df = get_off_stats('../data/pbp-2016.csv')
#df = df.append(get_off_stats('../data/pbp-2015.csv'))
#df = df.append(get_off_stats('../data/pbp-2014.csv'))

In [17]:
conn = sqlite3.connect('fantasy.db')
c = conn.cursor()

In [9]:
df.head()

,Team,Season,Yards,Rushes,Passes,Incompletes,PassYrds,RushYrds,PassTDs,RushTDs,TDs,Plays
0,ARI,2016,5803,365,647,255,4487,1627,27,19,50,1324
1,ATL,2016,6456,393,526,159,4847,1865,36,19,55,1240
2,BAL,2016,5789,345,658,222,4478,1551,21,9,31,1311
3,BUF,2016,5695,458,472,180,3245,2647,18,29,47,1250
4,CAR,2016,5718,426,563,252,4215,1760,26,16,43,1293


In [18]:
c.execute('''CREATE TABLE OffenseSeason
             (Team VARCHAR(3), 
             Season CHARACTER(4), 
             Yards SMALLINT,
             Rushes SMALLINT,
             Passes SMALLINT,
             Incompletes SMALLINT,
             PassYrds SMALLINT,
             RushYrds SMALLINT,
             PassTDs SMALLINT,
             RushTDs SMALLINT,
             TDs SMALLINT,
             Plays SMALLINT,
             UNIQUE (Team, Season))''')

In [19]:
for ndx in range(len(df)):
    try:
        c.execute('INSERT INTO OffenseSeason VALUES (?,?,?,?,?,?,?,?,?,?,?,?)', tuple(df.iloc[ndx].astype('str'))) 
    except:
        print(str(df.iloc[ndx]['Team'])+' '+str(df.iloc[ndx]['Season'])+' already in table')

('ARI', '2016', '5803', '365', '647', '255', '4487', '1627', '27', '19', '50', '1324')
('ATL', '2016', '6456', '393', '526', '159', '4847', '1865', '36', '19', '55', '1240')
('BAL', '2016', '5789', '345', '658', '222', '4478', '1551', '21', '9', '31', '1311')
('BUF', '2016', '5695', '458', '472', '180', '3245', '2647', '18', '29', '47', '1250')
('CAR', '2016', '5718', '426', '563', '252', '4215', '1760', '26', '16', '43', '1293')
('CHI', '2016', '5918', '334', '573', '206', '4504', '1584', '19', '10', '31', '1163')
('CIN', '2016', '5606', '400', '563', '197', '4216', '1664', '17', '15', '32', '1254')
('CLE', '2016', '4567', '267', '557', '224', '3687', '1296', '14', '12', '29', '1172')
('DAL', '2016', '6124', '479', '473', '153', '3779', '2503', '27', '25', '52', '1244')
('DEN', '2016', '5186', '367', '575', '238', '3964', '1465', '20', '10', '31', '1250')
('DET', '2016', '5293', '331', '590', '209', '4219', '1306', '26', '9', '36', '1190')
('GB', '2016', '5706', '338', '614', '219', '

In [22]:
conn.commit()

In [28]:
conn.close()